In [10]:
import cv2
import time
import math
import carla
import weakref
import numpy as np
import threading
import time
import random

In [11]:
# Global dictionary to store GNSS data
gnss_data = {'lat': None, 'lon': None}
stop_flag = threading.Event()
log_file = "gnss_coordinates.txt"  # File to save coordinates

In [2]:
import sys
sys.path.append('/home/kaustubh/CARLA_0_9_14/PythonAPI/carla')

In [3]:
client = carla.Client("localhost", 2000)

In [4]:
world = client.get_world()

In [5]:
map_ = world.get_map()

In [6]:
spawn_points = map_.get_spawn_points()

In [7]:
# Selecting the vehicle and spawning it
# vehicle_bp = world.get_blueprint_library().filter('vehicle.mini.cooper_s')
vehicle = world.get_actors().filter('*vehicle*')[0]
start_point = spawn_points[0]
# vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [8]:
print(vehicle)

Actor(id=543, type=vehicle.volkswagen.t2_2021)


In [30]:
# Camera setup
# import carla.libcarla


# camera_pos_z = 3
# camera_pos_x = 0
# camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
# # camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
# camera_bp.set_attribute('image_size_x', '640')
# camera_bp.set_attribute('image_size_y', '360')

# camera_init_trans = carla.Transform(carla.Location(z=camera_pos_z, x=camera_pos_x))
# camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# def camera_callback(image, data_dict):
#     data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# image_w = camera_bp.get_attribute('image_size_x').as_int()
# image_h = camera_bp.get_attribute('image_size_y').as_int()

# camera_data = {'image': np.zeros((image_h, image_w, 4))}
# camera.listen(lambda image: camera_callback(image, camera_data))

In [31]:
# PREFERRED_SPEED = 30
# SPEED_THRESHOL = 2

# font = cv2.FONT_HERSHEY_SIMPLEX

# org = (30, 30)
# org2 = (30, 50)
# org3 = (30, 70)
# org4 = (30, 90)
# org5 = (30, 110)

# fontscale = 0.5

# color = (255, 255, 255)
# thickness = 1

# def maintain_spped(s):
#     if s >= PREFERRED_SPEED:
#         return 0
#     elif s < PREFERRED_SPEED - SPEED_THRESHOL:
#         return 0.8
#     else:
#         return 0.4

In [28]:
import carla.libcarla


camera_pos_z = 2.5
camera_pos_x = 0.65
# camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp = world.get_blueprint_library().find('sensor.camera.instance_segmentation')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=camera_pos_z, x=camera_pos_x))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

quit = False

while True: 
    world.tick()
    
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break
    
    image = camera_data['image']
    
    # steering_angle = 0
    # v = vehicle.get_velocity()
    # speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)
    # image = cv2.putText(image, 'Speed: ' + str(int(speed)) + ' kmh', org2,
    #                     font, fontscale, color, thickness, cv2.LINE_AA)
    
    # estimate_throttle = maintain_spped(speed)
    # estimate_throttle = 10
    # vehicle.apply_control(carla.VehicleControl(throttle=estimate_throttle,
    #                                            steer=steering_angle))
    
    cv2.imshow('RGB Camera', image)
    
cv2.destroyAllWindows()
camera.stop()
# for actor in world.get_actors().filter('*vehicle*'):
#     actor.destroy()
# for sensor in world.get_actors().filter('*sensor*'):
#     sensor.destroy()

In [7]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [19]:
x = world.get_actors().filter('*vehicle*')

for i in x:
    print(i)

Actor(id=507, type=vehicle.volkswagen.t2_2021)


In [9]:
for sen in world.get_actors().filter('*gnss*'):
    print(sen)

Actor(id=546, type=sensor.other.gnss)


In [19]:
gnss = world.get_actors().filter('*gnss*')[0]

Waiting for GNSS data...
Waiting for GNSS data...
Waiting for GNSS data...


In [ ]:

def _on_gnss_event(event):
    # Update the global dictionary directly
    gnss_data['lat'] = event.latitude
    gnss_data['lon'] = event.longitude

# Function to log GNSS data to a text file every 4 seconds for 1 minute
def log_gnss_data():
    start_time = time.time()
    with open(log_file, "w") as file:  # Open file in write mode
        while not stop_flag.is_set():
            print
            current_time = time.time()
            elapsed = current_time - start_time

            # Stop after 1 minute
            if elapsed > 60:
                print("Logging complete.")
                break

            if gnss_data['lat'] is not None and gnss_data['lon'] is not None:
                file.write(f"{gnss_data['lat']:.7f},{gnss_data['lon']:.7f}\n")
                file.flush()  # Immediately write to disk
                print(f"Logged: Latitude: {gnss_data['lat']}, Longitude: {gnss_data['lon']}")
            else:
                print("Waiting for GNSS data...")

            time.sleep(4)

# Start the logging thread
thread = threading.Thread(target=log_gnss_data, daemon=True)
thread.start()

# Assuming 'gnss' is your actual GNSS object
# gnss.listen(_on_gnss_event)  # Uncomment when using a real GNSS object

# Function to choose a random coordinate from the file


Logged: Latitude: 0.0002904473135885155, Longitude: -0.0009534886969418


Logged: Latitude: 0.0002934100913449811, Longitude: -0.000952634943776038
Logged: Latitude: 0.00029756926808488515, Longitude: -0.0009514279559778332
Logged: Latitude: 0.00048222160988586893, Longitude: -0.0008395302973369796
Logged: Latitude: 0.0005028655834848905, Longitude: -0.000806694006155453
Logged: Latitude: 0.0005655435531082276, Longitude: -0.00043122272587322956
Logged: Latitude: 0.0005659280058978311, Longitude: -0.00041824138739897154
Logged: Latitude: 0.000570594520510781, Longitude: -0.00025882327035398045
Logged: Latitude: 0.0005716884923856469, Longitude: -0.00022142916269127122
Logged: Latitude: 0.0005857133260178671, Longitude: 0.00025869793511259907
Logged: Latitude: 0.0005878151200562343, Longitude: 0.00033073607421345546
Logged: Latitude: 0.0003804216025429241, Longitude: 0.0008545270744678616
Logged: Latitude: 0.0003393935438822382, Longitude: 0.0008712295074578046
Logging complete.
Logged: Latitude: -0.0001164877384525198, Longitude: 0.0009677317834919736
Logged

In [ ]:
gnss.listen(_on_gnss_event)

Logged: Latitude: 0.0002904473135885155, Longitude: -0.0009534886969418
Logged: Latitude: 0.0002904473135885155, Longitude: -0.0009534886969418


In [26]:
def choose_random_coordinate():
    with open(log_file, "r") as file:
        lines = file.readlines()
        random_coordinate = random.choice(lines).strip()
        lat, lon = map(float, random_coordinate.split(","))
        print(f"Chosen Coordinate: Latitude: {lat}, Longitude: {lon}")
        return lat, lon


choose_random_coordinate()

Chosen Coordinate: Latitude: -0.0012633, Longitude: 1.12e-05


(-0.0012633, 1.12e-05)